<center><img src = "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.WmN3Js3CIU7Yy072-JKSEQHaDH%26pid%3DApi&f=1" width = 400></center>

# Segmenting and Clustering Neighborhoods in Toronto
Toronto is a beautiful city with many diverse neighborhoods. I will explore analyze them by scraping postal codes from Wikipedia and utilizing Foursquare's powerful API to derive additional insights.

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!pip install geocoder
import geocoder

!pip install bs4
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         375 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates

# Section 1: Scraping Postal Codes from Wikipedia

In [4]:
# Scraping text from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(url).text, 'html5lib')
tables = soup.find_all('table')
len(tables)  # Checking how many tables there are

3

In [5]:
# Getting details about the tables to identify the correct one
for i, table in enumerate(tables):
    print('Table ', i)
    print(table.attrs)
    print('# rows: ', len(table.find_all('tr')))
    print('# columns: ', len(table.tr.find_all('td')), '\n')

Table  0
{'style': 'width:100%; border-collapse:collapse; border:1px solid #ccc;', 'cellpadding': '2', 'cellspacing': '0', 'rules': 'all'}
# rows:  20
# columns:  9 

Table  1
{'class': ['navbox']}
# rows:  4
# columns:  2 

Table  2
{'cellspacing': '0', 'style': 'background-color: #F8F8F8;', 'width': '100%'}
# rows:  2
# columns:  12 



In [6]:
# Getting postal codes from the first table
pcTable = []  # Empty list to store table content

for cell in tables[0].find_all('td'):
    pcRow = {}
    if cell.span.text == 'Not assigned':  # Removing unnassigned codes
        pass
    else:
        pcRow['Postal Code'] = cell.p.text[:3]
        pcRow['Borough'] = (cell.span.text).split('(')[0]
        pcRow['Neighborhood'] = (((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        pcTable.append(pcRow)

boroughs = pd.DataFrame(pcTable)  # Creating dataframe
boroughs['Borough'] = boroughs['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto Stn A',  # Cleaning text
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto Business',
                                       'EtobicokeNorthwest': 'Etobicoke Northwest','East YorkEast Toronto': 'East York/East Toronto',
                                       'MississaugaCanada Post Gateway Processing Centre': 'Mississauga'})
boroughs.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [7]:
print('DataFrame size: ', boroughs.shape)

DataFrame size:  (103, 3)


# Section 2: Collecting Latitude and Longitude for Toronto Neighborhoods

Could never get geocoder to return a response, so I had to rely on the CSV provided in the hints.

In [8]:
gs_data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
gs_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
type(gs_data)  # disregard..kept getting 'Nonetype has no attribute __' errors

pandas.core.frame.DataFrame

In [10]:
type(boroughs)  # disregard..kept getting 'Nonetype has no attribute __' errors

pandas.core.frame.DataFrame

In [12]:
# Joining the data from the csv with the first dataframe on Postal Code column
df = boroughs.merge(gs_data, on='Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [13]:
print('New dataframe size: ', df.shape)

New dataframe size:  (103, 5)


# Section 3: Extracting Toronto Data From Foursquare API


In [14]:
# @hidden_cell
YOUR_CLIENT_ID = '2KHXI4MUXJD0V2K0YGFQTNQRCTTKKSRBWEP3N5QJ24SY51AX'
YOUR_CLIENT_SECRET = '3MWQPADE3QEFK2CSXOGWR5RP4UL13MBBAKMX1SXGYWXHKPZ4'
ACCESS_TOKEN = 'CJ122XDLMQN1WJHG2GSGYUPADDRMTBUSLIAN3HSK3HKLIO2Q'

In [15]:
VERSION = '20180605'
cred = '&client_id={}&client_secret={}&v={}'.format(YOUR_CLIENT_ID,
                                                   YOUR_CLIENT_SECRET,
                                                   VERSION)

#### Creating a formula to get venues and categories from each neighborhood:

In [16]:
def getVenues(neighs, latitudes, longitudes, radius=500, limit=500):
    venues = []
    for neigh, lat, lng in zip(neighs, latitudes, longitudes):
        print(neigh)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?'+cred+'&ll={},{}&radius={}&limit={}'.format(
            lat,
            lng,
            radius,
            limit)

        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues.append([(
            neigh,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venues in venues for item in venues])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']

    return(nearby_venues)

#### I chose to work with just neighborhoods in boroughs that had 'Toronto' in their name, so I created a new dataframe with just those:

In [17]:
t_neighs = []
for i, bor in enumerate(df['Borough']):
    t_data = {}
    if bor.find('Toronto') == -1:
        pass
    else:
        t_data['Postal Code'] = df['Postal Code'][i]
        t_data['Borough'] = bor
        t_data['Neighborhood'] = df['Neighborhood'][i]
        t_data['Longitude'] = df['Longitude'][i]
        t_data['Latitude'] = df['Latitude'][i]
        t_neighs.append(t_data)
tdf = pd.DataFrame(t_neighs)
tdf.head()

,Postal Code,Borough,Neighborhood,Longitude,Latitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",-79.360636,43.654260
1,M5B,Downtown Toronto,"Garden District, Ryerson",-79.378937,43.657162
2,M5C,Downtown Toronto,St. James Town,-79.375418,43.651494
3,M4E,East Toronto,The Beaches,-79.293031,43.676357
4,M5E,Downtown Toronto,Berczy Park,-79.373306,43.644771


In [18]:
print('Size: ', tdf.shape)

Size:  (39, 5)


In [19]:
# Running the formula on my new dataframe:
toronto_venues = getVenues(neighs=tdf['Neighborhood'],
                           latitudes=tdf['Latitude'],
                           longitudes=tdf['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [20]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,46,46,46,46,46,46
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,60,60,60,60,60,60
Christie,15,15,15,15,15,15
Church and Wellesley,66,66,66,66,66,66
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,28,28,28,28,28,28
Davisville North,9,9,9,9,9,9


In [22]:
print('There are {} venues total and {} unique venue categories.'.format(
    toronto_venues.shape[0],
    len(toronto_venues['Venue Category'].unique())))

There are 1511 venues total and 221 unique venue categories.


# Section 4: Neighborhood Analysis and Segmenting

In [29]:
# Doing some one-hot encoding with the categories
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#### After multiple errors, I discovered that one of the venue categories is 'Neighborhood'. So when I add my Neighborhood column back into the dataframe, I will need to add an underscore to differentiate them.

In [24]:
# Checking to see which venues have the neighborhood category, out of curiosity
for i, category in enumerate(toronto_venues['Venue Category']):
    if category.find('Neigh') == -1:
        pass
    else:
        print('{} - {}'.format(toronto_venues['Venue'][i], category))

Upper Beaches - Neighborhood
Downtown Toronto - Neighborhood
Harbourfront - Neighborhood
Leslieville - Neighborhood


In [30]:
# Setting Neighborhood_ to the first column
fixed_columns = ['Neighborhood_'] + list(toronto_onehot.columns[::])

# Adding Neighborhood_
toronto_onehot['Neighborhood_'] = toronto_venues['Neighborhood']

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood_,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
print('Onehot dataframe size: ', toronto_onehot.shape)

Onehot dataframe size:  (1511, 223)



#### Grouping by Neighborhood_ and getting the frequency for each category:

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood_').mean().reset_index()
toronto_grouped.head()

,Neighborhood_,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.021739,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.021739,0.065217,0.000000,0.000000,0.0,0.021739,0.021739,0.0,0.043478,0.0,0.0,0.021739,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.021739,0.0,0.021739,0.0,0.0,0.0,0.0,0.021739,0.0,0.0,0.0,0.000000,0.0,0.086957,0.086957,0.0,0.0,0.0,0.0,0.021739,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.021739,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.043478,0.000000,0.0,0.0,0.0,0.0,0.0,0.021739,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021739,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.021739,0.0,0.000000,0.021739,0.021739,0.0

In [32]:
print('Grouped dataframe size is ', toronto_grouped.shape)

Grouped dataframe size is  (39, 222)


#### Creating the function to sort venues in descending order:

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating a dataframe of the top ten categories per neighborhood, based on the frequencies above:

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# New dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood_']

# Getting indicator values
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Sandwich Place,Vegetarian / Vegan Restaurant,Farmers Market,Beer Bar,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Breakfast Spot,Coffee Shop,Café,Sandwich Place,Convenience Store,Restaurant,Nightclub,Japanese Restaurant,Italian Restaurant,Bar
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Coffee Shop,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Café,Italian Restaurant,Japanese Restaurant,Burger Joint,Salad Place,Bank,Pizza Place
4,Christie,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Coffee Shop,Comfort Food Restaurant


In [35]:
print('Size: ', toronto_venues_sorted.shape)

Size:  (39, 11)


# Section 5: Neighborhood Clustering

#### Creating a K-means clustering model for the data:

In [40]:
# Making 5 clusters
k = 5

# Removing string values from the table
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_', 1)

# Fitting the model
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

# Double-checking the labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Inserting cluster information into a final dataframe:

In [41]:
# Adding the labels to the dataframe
toronto_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

# Joining with the Toronto ('tdf') dataframe to get lat/long for each neighborhood
toronto_merged = tdf.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Longitude,Latitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",-79.360636,43.654260,0,Coffee Shop,Park,Pub,Bakery,Café,Restaurant,Performing Arts Venue,Dessert Shop,Chocolate Shop,Discount Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",-79.378937,43.657162,0,Coffee Shop,Sandwich Place,Clothing Store,Café,Pizza Place,Bank,Hotel,Cosmetics Shop,Japanese Restaurant,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,-79.375418,43.651494,0,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Clothing Store,Gastropub,Restaurant,American Restaurant,Moroccan Restaurant,Farmers Market
3,M4E,East Toronto,The Beaches,-79.293031,43.676357,3,Health Food Store,Playground,Neighborhood,Pub,Yoga Studio,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store
4,M5E,Downtown Toronto,Berczy Park,-79.373306,43.644771,0,Coffee Shop,Cocktail Bar,Bakery,Sandwich Place,Vegetarian / Vegan Restaurant,Farmers Market,Beer Bar,Seafood Restaurant,Italian Restaurant,Japanese Restaurant


In [42]:
print('Merged dataframe size: ', toronto_merged.shape)

Merged dataframe size:  (39, 16)


#### Generating the cluster map:

In [46]:
# Did a google search to get Toronto coords since geocoder wasn't working (ironic)
latitude = 43.7001100
longitude = -79.4163000

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters